In [1]:
!pip install langchain
!pip install langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.8 MB/s eta 0:00:00


In [2]:

from langchain_core.prompts import ChatPromptTemplate,FewShotChatMessagePromptTemplate

In [10]:
pip install -qU langchain-groq



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 5.9 MB/s eta 0:00:00


In [19]:
from langchain_groq import ChatGroq
from langchain.output_parsers import DatetimeOutputParser

llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
import os
os.environ['GROQ_API_KEY'] = "gsk_29zKh0vydWb6pcOTmRWpWGdyb3FYKEtoXXXOXdBt3000000"


In [25]:
#example
datetime = DatetimeOutputParser()
print(datetime.get_format_instructions()) #dataetime

Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.

Examples: 1327-08-12T15:44:04.390907Z, 1848-03-07T20:43:06.740646Z, 1864-09-22T11:25:25.644279Z

Return ONLY this string, no other words!


In [26]:
datetime = DatetimeOutputParser()
human_template = "{request}\n{format_instruction}"
chat_prompt = ChatPromptTemplate.from_template(human_template)

In [34]:
from langchain.output_parsers import CommaSeparatedListOutputParser
csp = CommaSeparatedListOutputParser()

In [37]:
format = chat_prompt.format_messages(request="What date when ww2 was started",format_instruction=csp.get_format_instructions())

In [39]:
print(format)

[HumanMessage(content='What date when ww2 was started\nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`', additional_kwargs={}, response_metadata={})]


In [41]:
response = llm.invoke(format)
print("Response Content: ",response)
print("parser result is: ",csp.parse(response.content))

Response Content:  content='September 1, 1939' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 48, 'total_tokens': 58, 'completion_time': 0.016349163, 'prompt_time': 0.003714529, 'queue_time': 0.010714498000000001, 'total_time': 0.020063692}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None} id='run-29fedd5e-5727-4748-8073-a1ebf309ea1e-0' usage_metadata={'input_tokens': 48, 'output_tokens': 10, 'total_tokens': 58}
parser result is:  ['September 1', '1939']


In [43]:
# example
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel,Field

In [51]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers.pydantic import PydanticOutputParser
from typing import List

class Cricketer(BaseModel):
    name: str = Field(description="Name of the cricketer")
    records: List[str] = Field(description="List of records for the cricketer")


parser = PydanticOutputParser(pydantic_object=Cricketer)

print(parser.get_format_instructions())


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "Name of the cricketer", "title": "Name", "type": "string"}, "records": {"description": "List of records for the cricketer", "items": {"type": "string"}, "title": "Records", "type": "array"}}, "required": ["name", "records"]}
```


In [52]:
human_temp = "{request}\n{formate_instruction}"

In [53]:
formatchat = ChatPromptTemplate.from_template(human_temp)

In [54]:
formatted = formatchat.format_messages(request="Who is the best cricketer in the world",formate_instruction=parser.get_format_instructions())

In [56]:
response = llm.invoke(formatted)
print("reponse is",response.content)
print("parser",parser.parse(response.content))

reponse is {
"name": "Virat Kohli",
"records": [
"Highest score in T20I cricket (244 runs against West Indies in 2018)",
"Fastest batsman to 10,000 ODI runs (in 205 innings)",
"First batsman to score 500 or more runs in four consecutive IPL seasons",
"Most runs scored in a single calendar year in ODI cricket (2018 - 2,354 runs)"
]
}
parser name='Virat Kohli' records=['Highest score in T20I cricket (244 runs against West Indies in 2018)', 'Fastest batsman to 10,000 ODI runs (in 205 innings)', 'First batsman to score 500 or more runs in four consecutive IPL seasons', 'Most runs scored in a single calendar year in ODI cricket (2018 - 2,354 runs)']
